# **Measuring Bias with the aif360 library**


In order to make sure that your algorithm treats everyone equally, it is important to measure the bias. Here, we will illustrate how to measure bias in a binary classification context. 

Let's imagine that a company is looking to hire a new employee. They use a machine learning algorithm to select the top candidates. The candidates are assigned either 0 if they are not selected or 1 if they are. 

In this notebook, we will:
1. Install and import useful modules, including the aif360 library, and load the data
2. Transform the data in the aif360 format
3. Train and evaluate a ML model, and obtain predictions for the test set
4. Create a ClassificationMetric object and calculate accuracy and bias metrics

We will implement the ridge classifier in [scikit-learn](https://scikit-learn.org/stable/), and we will use the [aif360](https://aif360.readthedocs.io/en/latest/index.html) library to quantify the bias.



## **Import aif360 and other modules, load the data**

In [ ]:
#Imports 

import pickle
import numpy as np
import pandas as pd

In [ ]:
#Install and import aif360

!pip install aif360 
import aif360

Please download the data from the following link: https://hai-data.s3.eu-west-2.amazonaws.com/roadmaps/data.pickle. If running in Colab, please upload the data to the local folder. Otherwise, place the data in the same folder as the notebook.



In [ ]:
# Only run if running on Google Colab
!pip3 install pickle5
import pickle5 as pickle
!gdown --id 1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg3R # download data file from public link and place it in content/ folder

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg3R 



In [ ]:
path = '' 
if 'google.colab' in str(get_ipython()):
  path = path + '/content/'
path = path + 'data.pickle'

#Load the data
with open(path, 'rb') as handle:
    raw_data = pickle.load(handle)  
raw_data[:5] #display the first 5 candidates data

,Label,Gender,Ethnicity,0,1,2,3,4,5,6,...,490,491,492,493,494,495,496,497,498,499
0,0,Female,White,28.021737,4.351153,2.453895,1.637143,-1.746628,-0.483463,0.034170,...,-0.557444,-0.015627,-0.052749,-0.234189,-0.072384,0.090403,0.376761,0.258914,-0.050558,0.014513
1,0,Female,White,29.603342,-3.407193,0.771800,-2.957411,0.599226,-2.805277,0.329414,...,-0.198440,-0.158843,0.191984,-0.004532,0.229210,-0.173042,-0.072871,0.442939,-0.054423,0.026959
2,1,Female,Hispanic,26.504283,0.642464,2.522944,-2.197094,2.270646,-0.472510,0.532815,...,0.423352,-0.033844,-0.125387,-0.483924,-0.116553,-0.113281,0.015519,0.017111,-0.012309,0.264572
3,0,Female,Hispanic,25.012088,0.895121,-2.092517,3.687830,0.539642,1.988930,1.121646,...,-0.280392,0.046582,0.116709,0.133876,0.072716,0.124083,0.213735,-0.149901,-0.217130,0.004403
4,1,Male,Hispanic,27.358934,-2.332423,0.154999,-2.623793,1.682456,1.262280,-1.685565,...,-0.019350,-0.093371,0.003443,-0.025467,0.155397,-0.067609,-0.084833,0.033429,-0.199198,0.229629


In [ ]:
# remove all nans --> we use the variable "data" in the rest of this notebook
data = raw_data.dropna()

## **Transform data in aif360 format**

We first split our data into train and test sets. We use the sklearn library to do this.

In [ ]:
from sklearn.model_selection import train_test_split

#Split data into train/test
data_train, data_test = train_test_split(data, test_size = 0.3, random_state=4)

We then  need to process our data so it is in the standard aif360 format. aif360 uses custom dataset classes. We will use the BinaryLabelDataset class for our problem. The BinaryLabelDataset has four components:
- Features : features used to train the model
- Labels : Outcome of binary classification
- Scores: Probability of outcome (not available in our case)
- Protected characteristics (*Gender* and *Ethnicity* columns for us)

We therefore need to add the columns "Gender" and "Ethnicity" to our dataset. 


In [ ]:
from aif360.datasets import BinaryLabelDataset
from sklearn.preprocessing import LabelEncoder

# make Gender and Ethnicity numerical
def encode(df):
  g_enc = LabelEncoder()
  e_enc = LabelEncoder()
  #add Gender and Ethnicity to the data
  df['Gender'] = g_enc.fit_transform(df['Gender'])
  df['Ethnicity'] = e_enc.fit_transform(df['Ethnicity'])
  return df, g_enc,e_enc

#Add Gender and Ethnicity to both datasets 
data_train, g_enc, e_enc = encode(data_train.copy())
data_test, _, _ = encode(data_test.copy())

#Print codes for the protected attributes
print("Gender codes - Male : %d - Female : %d"%(g_enc.transform(['Male'])[0], g_enc.transform(['Female'])[0]))
print("Ethnicity codes - White : %d - Black : %d - Asian : %d - Hispanic : %d"%(e_enc.transform(['White'])[0], e_enc.transform(['Black'])[0], e_enc.transform(['Asian'])[0], e_enc.transform(['Hispanic'])[0]))

Gender codes - Male : 1 - Female : 0
Ethnicity codes - White : 3 - Black : 1 - Asian : 0 - Hispanic : 2


We're now going to make two BinaryLabelDatasets: one for the training data and one for test data. See [documentation here](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.BinaryLabelDataset.html#aif360.datasets.BinaryLabelDataset) for BinaryLabelDataset and [here](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StructuredDataset.html#aif360.datasets.StructuredDataset) for StructuredDataset the base class on which the former is built. 

In [ ]:
# Make BinaryLabelDatasets 
# Notice that we have to specify the following arguments: *favorable_label, unfavorable_label,df,label_names,protected_attribute_names*.
ds_train_true = BinaryLabelDataset(favorable_label=1.0, unfavorable_label=0.0, df=data_train, label_names=['Label'], protected_attribute_names=['Gender','Ethnicity'])
ds_test_true = BinaryLabelDataset(favorable_label=1.0, unfavorable_label=0.0, df=data_test, label_names=['Label'], protected_attribute_names=['Gender','Ethnicity'])

By default, the protected attributes are used as features by the BinaryLabelDataset object. We will update the features so they don't contain the protected attributes. You can use *ds.feature_names* to check which columns are included.


In [ ]:
# Define function to split the data 
def split_data_from_df(data):
  y = data['Label'].values
  X = data[np.arange(500)].values
  filter_col = ['Ethnicity', 'Gender'] + [col for col in data if str(col).startswith('Ethnicity_')] + [col for col in data if str(col).startswith('Gender_')] 
  dem = data[filter_col].copy()
  return X, y, dem

# Update features so that they don't contain protected attributes
X_train, y_train, dem_train = split_data_from_df(data_train)
X_test, y_test, dem_test = split_data_from_df(data_test)
ds_train_true.features = X_train
ds_test_true.features = X_test

#Set the features to be the 500 pre-defined features, whose columns names are 0,1..499
ds_test_true.feature_names = np.arange(500).astype(str)
ds_train_true.feature_names = np.arange(500).astype(str)

## **Train the model and get the predictions for the test set**

We can now proceed to create a RidgeClassifier model with the sklearn library, train the model and get the predictions for the test set. We use *X_train=ds_train.features* and *y_train=ds_train.labels.ravel()* to access X and y from the dataset created earlier; and similarly for *X_test,y_test*.

In [ ]:
from sklearn.linear_model import RidgeClassifier

# create model
model = RidgeClassifier(random_state = 42)

# train model
X_train = ds_train_true.features
y_train = ds_train_true.labels.ravel()
model.fit(X_train,y_train)

X_test = ds_test_true.features
y_test = ds_test_true.labels.ravel()
y_test_pred = model.predict(X_test)

#print accuracy
print(model.score(X_test, y_test))

0.7133085323412937


## **Create a ClassificationMetric object and calculate accuracy and bias metrics**

We now need to choose what privileged and unprivileged groups we want to focus on. You can try different combinations by changing the attributes in the dropdown. 

In [ ]:
#@title Select privileged and unprivileged groups  { display-mode: "both" }

#text = 'value' #@param {type:"string"}
protected_attribute = 'Ethnicity' #@param ["Ethnicity", "Gender"]

privileged_group = 'White' #@param ["Asian", "Black", "Hispanic", "White", "Male", "Female"]
unprivileged_group = 'Black' #@param ["Asian", "Black", "Hispanic", "White", "Male", "Female"]



In [ ]:
#Find codes for each group 
if protected_attribute == 'Ethnicity':
  code_privileged_group = e_enc.transform([privileged_group])[0]
  code_unprivileged_group = e_enc.transform([unprivileged_group])[0]

if protected_attribute == 'Gender':
  code_privileged_group = g_enc.transform([privileged_group])[0]
  code_unprivileged_group = g_enc.transform([unprivileged_group])[0]

#Define privileged and unprivileged groups 
unprivileged_groups = [{protected_attribute:code_unprivileged_group}]  #list of privileged groups with the respective code
privileged_groups = [{protected_attribute:code_privileged_group}] #list of unprivileged groups with the respective code

We next have to create a [ClassificationMetric](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html) object for the test set. 
Notice that it takes 4 arguments: 
1. dataset: ds_test = the test dataset you created above 
2. classified_dataset: ds_test_pred = the same dataset but replacing the labels with the predictions as such:
    ```python
            ds_test_pred = ds_test.copy()
            ds_test_pred.labels = y_pred_test
    ```
3. unprivileged_groups
4. privileged_groups
    

In [ ]:
from aif360.metrics import ClassificationMetric

# create pred dataset by changing labels
ds_test_pred = ds_test_true.copy()
ds_test_pred.labels = y_test_pred

# create metric class
metric_ethnicity = ClassificationMetric(
        ds_test_true, ds_test_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)

We can now calculate and print the following metrics: 

1. Accuracy
2. Statistical Parity Difference
3. Disparate Impact
4. Equal Opportunity Difference


You can refer to the [roadmap](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html) for the definition of metrics.
 

In [ ]:
# evaluate
print("Accuracy : %.2f" %metric_ethnicity.accuracy())
print("Statistical Parity Difference : %.2f" %metric_ethnicity.statistical_parity_difference())
print("Disparate Impact : %.2f" %metric_ethnicity.disparate_impact())
print("Equal Opportunity Difference : %.2f" %metric_ethnicity.equal_opportunity_difference())

Accuracy : 0.71
Statistical Parity Difference : -0.09
Disparate Impact : 0.75
Equal Opportunity Difference : -0.11


## **Final remarks**

Note that in this example we only obtained one number for fairness metrics. In general, you should calculate a confidence interval by repeating the experiments for different train/test splits. A way to do this is for instance to use a non-parametric bootstrap method as described in section 10.2 from the book [Computer Age Statistical Inference by B.Efron and T.Hastie](https://web.stanford.edu/~hastie/CASI_files/PDF/casi.pdf). 